In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import DataLoader, WeightedRandomSampler, Dataset
import torchvision.datasets as datasets
import torchvision.transforms as transforms 
from tqdm.notebook import tqdm
import os
device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#hyperparameter
lr = 0.001
batch_size = 128
input_size = (80,80)
num_classes = 5
input_channel = 3
epochs = 50

In [3]:
class FlowerDataset(Dataset):
    def __init__(self, root_dir, weights):
        self.root_dir = root_dir
        self.transform = transforms.Compose(
            [
            transforms.Resize((64,64)),
            transforms.ToTensor()
            ]
        )
        dataset = datasets.ImageFolder(root=root_dir, transform= self.transform)
        self.dataset = []
        self.total = len(dataset)
        self.weights_count = [0]*5
        for (data, label) in tqdm(dataset):
            if self.weights_count[label] < self.total*weights[label]/5:
                self.dataset.append((data, label))
                self.weights_count[label]+=1
        

    def __len__(self):
        return sum(self.weights_count)

    def __getitem__(self, index):
        (X,y) = self.dataset[index]
        return (X, y)

root_dir = "newFlowers"
train_dataset = FlowerDataset(root_dir,[1/16,1/8,1/4,1/2,1])

train_set, _ = torch.utils.data.random_split(train_dataset, [int(0.8*len(train_dataset)), len(train_dataset)-int(0.8*len(train_dataset))])


test_dataset = FlowerDataset(root_dir,[1,1,1,1,1])

_, test_set = torch.utils.data.random_split(test_dataset, [int(0.8*len(test_dataset)), len(test_dataset)-int(0.8*len(test_dataset))])


In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

In [5]:
def get_mean_std(loader):
    # var[X] = E[X**2] - E[X]**2
    channels_sum, channels_sqrd_sum, num_batches = 0, 0, 0

    for data, _ in tqdm(loader):
        channels_sum += torch.mean(data, dim=[0, 2, 3])
        channels_sqrd_sum += torch.mean(data ** 2, dim=[0, 2, 3])
        num_batches += 1

    mean = channels_sum / num_batches
    std = (channels_sqrd_sum / num_batches - mean ** 2) ** 0.5

    return mean, std


mean, std = get_mean_std(train_loader)
print(mean)
print(std)
# mean : [0.4484, 0.4135, 0.3016]
#  std : [0.2841, 0.2545, 0.2773]


tensor([0.4650, 0.4047, 0.2806])
tensor([0.2802, 0.2335, 0.2482])


In [6]:
import torchvision
class CNN(nn.Module):
    def __init__(self, input_channel, num_classes):
        super(CNN, self).__init__()
        self.fc1 = nn.Linear(5,5)
        

    def forward(self, x):
        x = self.fc1(x)
        return x


# Simple Identity class that let's input pass without changes
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x


# Load pretrain model & modify it
model = torchvision.models.vgg16(pretrained=True)


# for param in model.parameters():
#     param.requires_grad = False

# If you want to do finetuning then set requires_grad = False
# Remove these two lines if you want to train entire model,
# and only want to load the pretrain weights.
# for param in model.parameters():
#     param.requires_grad = False
# model.avgpool = Identity()

model.classifier[-1] = nn.Linear(4096, 5)
model.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/dam797/.cache/torch/hub/checkpoints/vgg16-397923af.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# model = CNN(input_channel, num_classes).to(device)
criterion = nn.CrossEntropyLoss(weight=torch.tensor([1,1,1,1,1], dtype=torch.float32).to(device))
optimizer = optim.Adam(model.parameters(), lr = lr)

In [8]:
loss_per_epoch = []
for epoch in range(1, epochs):
    loop = tqdm(enumerate(train_loader), leave= False)
    loss_sum = 0.0 
    count = 0
    for batch_idx, (data, targets) in loop:

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
       
        ######
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        loss_sum+=loss
        count+=1
        # gradient descent or adam step
        optimizer.step()
       
        loop.set_description(f"Epoch [{epoch}/{epochs}]")
        loop.set_postfix(loss=torch.rand(1).item())

    loss_per_epoch.append(loss_sum/count)

In [9]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)


    model.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(test_loader, model)*100:.2f}")


Accuracy on training set: 21.23


# Weighted Loss


In [10]:
# Load pretrain model & modify it
model1 = torchvision.models.vgg16(pretrained=True)

model1.classifier[-1] = nn.Linear(4096, 5)
model1.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [11]:
# model = CNN(input_channel, num_classes).to(device)
criterion = nn.CrossEntropyLoss(weight=torch.tensor([16,8,4,2,1], dtype=torch.float32).to(device))
optimizer = optim.Adam(model1.parameters(), lr = lr)

In [12]:
loss_per_epoch1 = []
for epoch in range(1, epochs):
    loop = tqdm(enumerate(train_loader), leave= False)
    loss_sum = 0.0 
    count = 0
    for batch_idx, (data, targets) in loop:

        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model1(data)
       
        ######
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()
        loss_sum+=loss
        count+=1
        # gradient descent or adam step
        optimizer.step()
       
        loop.set_description(f"Epoch [{epoch}/{epochs}]")
        loop.set_postfix(loss=torch.rand(1).item())

    loss_per_epoch1.append(loss_sum/count)

In [13]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model1.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model1(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    model1.train()
    return num_correct/num_samples


print(f"Accuracy on training set: {check_accuracy(test_loader, model1)*100:.2f}")


Accuracy on training set: 36.95
